<a href="https://colab.research.google.com/github/MattyXarope/Comp-Sci-Homework-3-/blob/main/HypernymExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1026]:
# Import libraries required
import re
import nltk
import itertools
from nltk.tag.perceptron import PerceptronTagger
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1027]:
# Noun phrase chunking
chunk_patterns=r'''NP:{<DT>?<JJ.*>*<NN.*>+}
                      {<NN.*>+}
                '''
#chunk parser
nounphrase_chunker=nltk.RegexpParser(chunk_patterns)
# Hearst Patterns
hearst_patterns=[ (
                '(NP_\\w+ (, )?such as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(such NP_\\w+ (, )?as (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '((NP_\\w+ ?(, )?)+(and |or )?other NP_\\w+)',
                'last'
            ),
            (
                '(NP_\\w+ (, )?include (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                '(NP_\\w+ (, )?especially (NP_\\w+ ?(, )?(and |or )?)+)',
                'first'
            ),
            (
                    '((NP_\\w+ ?(, )?)+(and |or )?any other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?some other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be a NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?like (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    'such (NP_\\w+ (, )?as (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?like other NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of the NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of these NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?one of those NP_\\w+)',
                    'last'
                ),
                (
                    'example of (NP_\\w+ (, )?be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?be example of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?for example (, )?'
                    '(NP_\\w+ ?(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be call NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which be name NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?mainly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?mostly (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?notably (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?particularly (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?principally (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?in particular (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?except (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?other than (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?e.g. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ \\( (e.g.|i.e.) (, )?(NP_\\w+ ? (, )?(and |or )?)+'
                    '(\\. )?\\))',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?i.e. (, )?(NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? a kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? kind of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? form of NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which look like NP_\\w+)',
                    'last'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?which sound like NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which be similar to (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?example of this be (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?type (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )? NP_\\w+ type)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?whether (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(compare (NP_\\w+ ?(, )?)+(and |or )?with NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?compare to (NP_\\w+ ? (, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '(NP_\\w+ (, )?among -PRON- (NP_\\w+ ? '
                    '(, )?(and |or )?)+)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and |or )?as NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )? (NP_\\w+ ? (, )?(and |or )?)+ '
                    'for instance)',
                    'first'
                ),
                (
                    '((NP_\\w+ ?(, )?)+(and|or)? sort of NP_\\w+)',
                    'last'
                ),
                (
                    '(NP_\\w+ (, )?which may include (NP_\\w+ '
                    '?(, )?(and |or )?)+)',
                    'first'
                ),]

In [1028]:
# Part of Speech tagging
pos_tagger=PerceptronTagger()

In [1029]:
# Text Preprocessing
def prepare(raw_text):
    sentences=nltk.sent_tokenize(raw_text.strip())
    sentences=[nltk.word_tokenize(sent) for sent in sentences]
    sentences=[pos_tagger.tag(sent) for sent in sentences]
    return sentences

In [1030]:
def chunk(raw_text):
    sentences=prepare(raw_text.strip())
    all_chunks=[]
    for sentence in sentences:
        chunks=nounphrase_chunker.parse(sentence)
        all_chunks.append(prepare_chunks(chunks))
    all_sentences=[]
    for raw_sentence in all_chunks:
        sentence = re.sub(r"(NP_\w+ NP_\w+)+",lambda m: m.expand(r'\1').replace(" NP_", "_"),raw_sentence)
        all_sentences.append(sentence)
    return all_sentences

In [1031]:
# # Debug Chunk
# ex_corpus='I like music like rock and metal. I think countries like Iceland and Norway are beautiful. It is a good day to die.'
# sentences=prepare(ex_corpus)
# chunk=nounphrase_chunker.parse(sentences[0])
# chunk=prepare_chunks(chunk)
# re.sub(r"(NP_\w+ NP_\w+)+",lambda m: m.expand(r'\1').replace(" NP_", "_"),chunk)

In [1032]:
# Method to replace occurrences in text that are noun phrases and make them start with NP_<token> or just keep it as is
def prepare_chunks(chunks):
    terms=[]
    for chunk in chunks:
        label=None
        try:
            label=chunk.label()
        except:
            pass
        if label is None:
            token=chunk[0]
            terms.append(token)
        else:
            np='NP_'+'_'.join([a[0] for a in chunk])
            terms.append(np)
    return ' '.join(terms)

In [1033]:
# # Test preprocessing
# ex_corpus='I like music like rock and metal. I think countries like Iceland and Norway are beautiful. It is a good day to die.'
# sentences=nltk.sent_tokenize(ex_corpus)
# sentences=[nltk.word_tokenize(sent) for sent in sentences]
# sentences=[pos_tagger.tag(sent) for sent in sentences]
# chunk=nounphrase_chunker.parse(sentences[0])
# np_tagged_sentence=prepare_chunks(chunk)

In [1034]:
# # for chunko in chunk:
# #     print(chunko[1])
# # for (hearst_pattern,parser) in hearst_patterns:
# #     print(hearst_pattern)
# matches=re.search(list(hearst_patterns[8])[0],np_tagged_sentence)

In [1035]:
# match_str=matches.group(0)
# [a for a in match_str.split() if a.startswith('NP_')]

In [1036]:
def remove_np_term(term):
    return term.replace('NP_','').replace('_',' ')

In [1037]:
def find_hyponyms(rawtext):
    hypo_hypernyms=[]
    np_tagged_sentences=chunk(rawtext)

    for sentence in np_tagged_sentences:
        for (hearst_pattern,parser) in hearst_patterns:
            matches=re.search(hearst_pattern,sentence)
            if matches:
                match_str=matches.group(0)
                nps=[a for a in match_str.split() if a.startswith('NP_')]

                if parser=='first':
                    hypernym=nps[0]
                    hyponyms=nps[1:]
                else:
                    hypernym=nps[-1]
                    hyponyms=nps[:-1]
                for i in range(len(hyponyms)):
                    hypo_hypernyms.append((remove_np_term(hyponyms[i]),remove_np_term(hypernym)))
    return hypo_hypernyms

In [1038]:
# data=prepare(ex_corpus)
# all_chunks=[]

In [1039]:
# Apply Hearst Patterns to Wikipedia Files
def extractHearstWikipedia(input):
    extractions=[]
    lines_processed=0

    with open(input,'r', encoding='latin-1') as f:
        for  line in f:
            lines_processed+=1
            line=line.strip()
            if not line:
                continue
            line_split=line.split('\t')
            # sentence,lemma_sent=line_split[0].strip(),line_split[1].strip()
            sentence=line_split[0]
            hypo_hyper_pairs=find_hyponyms(sentence)
            extractions.append(hypo_hyper_pairs)

            if lines_processed%1000==0:
                print('Lines Processed: {}'.format(lines_processed))

    extractions=[x for x in extractions if x!=[]]
    extractions=list(itertools.chain.from_iterable(extractions))
    
    # hypo_hypernyms=[x for x in hypo_hyper_pairs if x!=[]]
    # hypo_hypernyms=list(itertools.chain.from_iterable(hypo_hypernyms))

    
    # with open('hypernyms','w') as f:
    #   for (hypo,hyper) in hypo_hypernyms:
    #     try:
    #       f.write(hypo+'\t'+hyper+'\n')
    #     except ValueError:
    #       pass       

    with open('subset_hypernyms','w') as f:
        for pair in extractions:
            try:
                f.write(str(pair)+'\n')
            except ValueError:
                pass

    return extractions

In [1040]:
# def extractHearstWikipedia(input):
#     extractions=[]
#     lines_processed=0

#     with open(input,'r') as f:
#         for  line in f:
#             lines_processed+=1
#             line=line.strip()
#             if not line:
#                 continue
#             line_split=line.split('\t')
#             # Uncomment to run on full dataset
#             # Changed to work on subset of the data
#             # sentence,lemma_sent=line_split[0].strip(),line_split[1].strip()
#             sentence=line_split[0]
#             hypo_hyper_pairs=find_hyponyms(sentence)
#             extractions.append(hypo_hyper_pairs)

#             if lines_processed%1000==0:
#                 print('Lines Processed: {}'.format(lines_processed))

#     hypo_hypernyms=[x for x in hypo_hyper_pairs if x!=[]]
#     hypo_hypernyms=list(itertools.chain.from_iterable(hypo_hypernyms))

    
#     with open('hypernyms','w') as f:
#       for (hypo,hyper) in hypo_hypernyms:
#         try:
#           f.write(hypo+'\t'+hyper+'\n')
#         except ValueError:
#           pass     
#     return extractions

In [1041]:
path = "/content/English Test 2.txt"
hypo_hyper_pairs=extractHearstWikipedia(path)

In [1042]:
x = hypo_hyper_pairs
hyponyms, hypernyms = zip(*x)

print(hyponyms)
print(hypernyms)


('the time', 'such sanctified names', 'Macaulay', 'Shakespeare', 'Grotius', 'Francis Bacon', 'Thomas Hobbes', 'universities', 'academies', 'a philosophy')
('bears', 'Charles', 'Hallam', 'Names', 'Names', 'Names', 'Names', 'institutions', 'institutions', 'anything')


In [1043]:
# Method to post process extracted noun pairs as hypernyms and hyponyms
# They contain multi-word noun phrases 
# We bring it down to the head pf the NP which is usually the last token and lemmatize
from nltk.stem.wordnet import WordNetLemmatizer
lemma=WordNetLemmatizer()
def post_process(text):
    text=text.split()
    length=len(text)
    return lemma.lemmatize(text[length-1])


In [1044]:
hyponyms_new=[]
for i in hyponyms:
  hyponyms_new.append(post_process(i.lower()))

hypernyms_new=[]  
for i in hypernyms:
  hypernyms_new.append(post_process(i.lower()))  

print(hyponyms_new)
print(hypernyms_new)

full_list = zip(hyponyms_new, hypernyms_new)
hypohyper_hearst_patterns = dict(full_list)
print(hypohyper_hearst_patterns)


['time', 'name', 'macaulay', 'shakespeare', 'grotius', 'bacon', 'hobbes', 'university', 'academy', 'philosophy']
['bear', 'charles', 'hallam', 'name', 'name', 'name', 'name', 'institution', 'institution', 'anything']
{'time': 'bear', 'name': 'charles', 'macaulay': 'hallam', 'shakespeare': 'name', 'grotius': 'name', 'bacon': 'name', 'hobbes': 'name', 'university': 'institution', 'academy': 'institution', 'philosophy': 'anything'}


In [1045]:
# # Convert the Extractions from Wikipedia into a dictionary
# hyperhypo={}
# with open('subset_hypernyms','r') as f:
#     text=f.read().strip().split('\n')
#     for line in text:
#         hyponym,hypernym=line.split('\t')   
#         hyponym,hypernym=hyponym.lower(),hypernym.lower()
#         hyponym=post_process(hyponym)
#         hypernym=post_process(hypernym)
#         hyperhypo[hypernym]=hyponym

In [1046]:
hypohyper_hearst_patterns

{'academy': 'institution',
 'bacon': 'name',
 'grotius': 'name',
 'hobbes': 'name',
 'macaulay': 'hallam',
 'name': 'charles',
 'philosophy': 'anything',
 'shakespeare': 'name',
 'time': 'bear',
 'university': 'institution'}

In [1047]:
# # Extract from BLESS 2011 data train file
# with open('/content/data_lex_train.tsv','r') as f:
#     inputlines=f.read().strip().split('\n')
# predictions=[]

# for line in inputlines:
#     split=line.split('\t')
#     hypo=split[0]
#     hyper=split[1]
#     if hyper not in hyperhypo:
#         predictions.append('{}\t{}\tFalse'.format(hypo,hyper))
#     else:
#         predictions.append('{}\t{}\tTrue'.format(hypo,hyper))

        
# with open('train_pred.tsv','w') as f:
#     for line in predictions:
#         f.write(line)
#         f.write('\n')

In [1065]:
# Extract Task training file
with open('/content/1A.english.training.data.txt','r') as f:
    inputlines=f.read().strip().split('\n')
hyponyms_task_training = inputlines

with open('/content/1A.english.training.gold.txt','r') as f:
    inputlines2=f.read().strip().split('\n')
hypernyms_gold_task_training = []
for line in inputlines2:
    split=line.split('\t')
    hypo=split[0:]
    hypernyms_gold_task_training.append(hypo)

taskdata = zip(hyponyms_task_training, hypernyms_gold_task_training)
hypohyper_training_data = dict(taskdata)
print(hypohyper_training_data)




# predictions=[]

# for line in inputlines:
#     split=line.split('\t')
#     hypo=split[0]
#     hyper=split[1]
#     if hyper not in hyperhypo:
#         predictions.append('{}\t{}\tFalse'.format(hypo,hyper))
#     else:
#         predictions.append('{}\t{}\tTrue'.format(hypo,hyper))

        
# with open('train_pred.tsv','w') as f:
#     for line in predictions:
#         f.write(line)
#         f.write('\n')

{'blackfly': ['homopterous insect', 'insect'], 'Turonian': ['technical specification', 'geologic timescale', 'physical property', 'geological period', 'magnitude', 'unit of time', 'geological time', 'geologic time'], 'abhorrence': ['distaste', 'hatred', 'hate', 'disgust'], 'tropical storm': ['atmosphere', 'windstorm', 'violent storm', 'air current', 'atmospheric state', 'density', 'current of air', 'storm damage', 'atmospheric phenomenon', 'storm', 'cyclone', 'natural phenomenon', 'tempest', 'wind'], 'militarization': ['social control'], 'pollution': ['dirtiness', 'dirtying', 'environmental condition', 'impurity', 'sanitary condition', 'uncleanness'], 'photomicrograph': ['picture', 'photograph', 'photo', 'image', 'pic', 'digital image'], 'swamp gum': ['eucalyptus', 'plant', 'eucalyptus tree', 'angiosperm', 'gum', 'gum tree', 'woody plant', 'eucalypt'], 'song': ['possession', 'sound', 'work of art', 'musical composition', 'vocal music', 'musical work', 'piece of music', 'human language'

In [1048]:
# function to read in from truth values and the predictions on trainset
def read_pred(file):
    labels=[]
    with open(file,'r') as f:
        data=f.read().strip().split('\n')
    for line in data:
        hypo,hyper,label=line.split('\t')
        if label=='True':
            labels.append(1)
        else:
            labels.append(0)

    return labels
truth_values=read_pred('/content/data_lex_train.tsv')
predicted_values=read_pred('train_pred.tsv')

# Precision, Recall and F1 Score
from sklearn.metrics import classification_report
classification_report(truth_values,predicted_values)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print('BLESS 2011 Data \n')
print('Precision: {}'.format(precision_score(truth_values,predicted_values)))
print('Recall: {}'.format(recall_score(truth_values,predicted_values)))
print('F1 Score: {}'.format(f1_score(truth_values,predicted_values)))


BLESS 2011 Data 

Precision: 0.20388349514563106
Recall: 0.54
F1 Score: 0.2960062646828504


In [1062]:
with open('/content/1A.english.training.gold.txt','r') as f:
    inputlines=f.read().strip().split('\n')

hypo_of_gold = []
for line in inputlines:
    split=line.split('\t')
    hypo=split[0:]
    hypo_of_gold.append(hypo)

print(hypo_of_gold)

[['homopterous insect', 'insect'], ['technical specification', 'geologic timescale', 'physical property', 'geological period', 'magnitude', 'unit of time', 'geological time', 'geologic time'], ['distaste', 'hatred', 'hate', 'disgust'], ['atmosphere', 'windstorm', 'violent storm', 'air current', 'atmospheric state', 'density', 'current of air', 'storm damage', 'atmospheric phenomenon', 'storm', 'cyclone', 'natural phenomenon', 'tempest', 'wind'], ['social control'], ['dirtiness', 'dirtying', 'environmental condition', 'impurity', 'sanitary condition', 'uncleanness'], ['picture', 'photograph', 'photo', 'image', 'pic', 'digital image'], ['eucalyptus', 'plant', 'eucalyptus tree', 'angiosperm', 'gum', 'gum tree', 'woody plant', 'eucalypt'], ['possession', 'sound', 'work of art', 'musical composition', 'vocal music', 'musical work', 'piece of music', 'human language', 'speech', 'signaling'], ['emblem', 'technical specification', 'form', 'specifications', 'symbolization', 'aerofoil', 'air uni

### Applying SVD to Hearst Patterns
Facebook AI paper [Hearst Patterns Revisited](https://research.fb.com/wp-content/uploads/2018/07/Hearst-Patterns-Revisited-Automatic-Hypernym-Detection-from-Large-Text-Corpora.pdf?) shows that pattern based methods outperform other methods. But extracting using regular Hearst Patterns ofteen leads to the problem of skewness due to cooccurence probabilities of the constituent words. For ex. It is likely for France and Country to occur together but not for France and Republic. Therefore we will try to implement the PPMI and truncated SVD of the PPMI matrix for Hearst Patterns


In [1050]:
# PPMI: Positive Pointwise Mutual Infomation of Hearst Patterns Revisited
# Coocurrence Matrix
with open('wikipedia_sentences.txt','r') as f:
    data=f.read().strip()

# Remove Stopwords and punct
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
tokenizer=RegexpTokenizer(r'\w+')
data=tokenizer.tokenize(data)
data=[word for word in data if word not in stopwords.words('english')]

FileNotFoundError: ignored

# New Section

In [ ]:
# Memory Trouble: Reduce Corpus size to contain memory
with open('wikipedia_sentences.txt','r') as f:
    data=f.read().strip()

In [ ]:
# Take a subset of the words in the dataset
# Find Hypernym pairs and extract PMI on top of them
data=sent_tokenize(data)
data=data[:50000]
data=' '.join(data)
with open('data_subset.txt','w') as f:
        f.write(data)

In [ ]:
hypo_hyper_pairs=extractHearstWikipedia('data_subset.txt')

In [ ]:
hyper_hyponums=[x for x in hypo_hyper_pairs if x!=[]]

In [ ]:
# Write pairs to file
import itertools
hyper_hyponums=list(itertools.chain.from_iterable(hyper_hyponums))
with open('subset_hypernyms.txt','w') as f:
    for (hypo,hyper) in hyper_hyponums:
        try:
            f.write(hypo+'\t'+hyper+'\n')
        except ValueError:
            pass

In [ ]:
# Dictionary from subset of pairs
hyperhypo={}
with open('subset_hypernyms.txt','r') as f:
    text=f.read().strip().split('\n')
    for line in text:
        hyponym,hypernym=line.split('\t')
        hyponym,hypernym=hyponym.lower(),hypernym.lower()
        hyponym,hypernym=post_process(hyponym),post_process(hypernym)
        hyperhypo[hypernym]=hyponym

In [ ]:
# Data: data_subset.txt
# Hypernym pairs: subset_hypernyms.txt
# Extract Frequencies of words from the dataset


In [ ]:
# Create Vocabularies and Dictionaries for lookups
vocab=set(data)
idx2word={key:value for key,value in zip(range(len(vocab)),vocab)}
word2idx={key:value for value,key in idx2word.items()}

In [ ]:
# Coocurrence Matrix
# Bigram COO matrix for Wikipedia Data
import numpy as np
def create_coo_matrix(data,idx2word,word2idx):
    coo_matrix=np.array()
    for word in data:
        coo_matrix.

In [ ]:
# Dep paths. Extract the noun pairs and the dep paths between them in the wikipedia data
# spacy to extract dependence paths and networkx library to extract the shortest path between those two noun pairs

# Test
import spacy
nlp=spacy.load('en_core_web_sm')

import networkx as nx
test_sentence='I like music like rock and metal.'
doc=nlp(test_sentence)
# Shortest path between two words
edges=[]
for token in doc:
    for child in token.children:
        edges.append(('{0}'.format(token.lower_),('{0}'.format(child.lower_))))
graph=nx.Graph(edges)

# Words to get deppaths for: music and rock or metal
word1='music'
word2='rock'
print(nx.shortest_path_length(graph,source=word1,target=word2))
print(nx.shortest_path(graph,source=word1,target=word2))

In [ ]:
extractions=[]
for sentence in data:
    extractions.append(find_hyponyms(sentence))
